In [ ]:
 pip install wandb transformers datasets bitsandbytes peft evaluate scikit-learn bert_score

In [ ]:
import os
import torch
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, TaskType

# 0. 메모리 단편화 완화
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

# 1. Quantization 설정
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# 2. 양자화된 SOLAR 모델 로드
model_name = "upstage/SOLAR-10.7B-v1.0"
base = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto"
)

# 3. LoRA 어댑터 설정 & 부착
lora_cfg = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=16,
    lora_alpha=8,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"]
)
model = get_peft_model(base, lora_cfg)

# 4. 캐시 끄기 & (일단) gradient checkpointing 비활성화
model.config.use_cache = False
# model.gradient_checkpointing_enable()   # 메모리 여유 있으면 켜세요

# 5. **Train 모드 활성화** — 여기서 반드시 호출해야 합니다!
model.train()

# 6. 학습 가능한 파라미터 수 출력 (디버그)
total, trainable = 0, 0
for n, p in model.named_parameters():
    num = p.numel()
    total += num
    if p.requires_grad:
        trainable += num
        print(f"[TRAINABLE] {n} ({num} params)")
print(f"▶︎ Trainable params: {trainable:,} / {total:,}")

# 7. 토크나이저 세팅
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({"additional_special_tokens": ["<|user|>", "<|assistant|>"]})
model.resize_token_embeddings(len(tokenizer), mean_resizing=False)

# 8. 데이터 로드 & 전처리
raw = load_dataset(
    "json",
    data_files={"train": "card_consult_finetune_messages.jsonl"},
    split="train"
)
MAX_LEN = 256

def format_data(example):
    # 같은 역할(role)이 연속된 메시지 합치기 + system 프롬프트 추가
    formatted = {"messages": []}
    formatted["messages"].append({
        "role": "system",
        "content": "당신은 친절하고 정확한 고객상담 챗봇입니다."
    })
    prev_role, temp = None, ""
    for msg in example["messages"]:
        role, content = msg["role"], msg["content"].strip()
        if role == prev_role:
            temp += " " + content
        else:
            if prev_role is not None:
                formatted["messages"].append({"role": prev_role, "content": temp})
            temp, prev_role = content, role
    if temp:
        formatted["messages"].append({"role": prev_role, "content": temp})
    return formatted

def preprocess(example):
    in_ids, lbls = [], []
    msgs = example["messages"]
    for i, msg in enumerate(msgs):
        if msg["role"] != "assistant":
            continue
        # build context
        ctx = ""
        for prev in msgs[:i]:
            tag = "<|user|>" if prev["role"]=="user" else "<|assistant|>"
            ctx += tag + prev["content"] + tokenizer.eos_token
        ctx += "<|assistant|>"
        # response
        resp = msg["content"] + tokenizer.eos_token
        # tokenize
        ctx_ids  = tokenizer(ctx,  add_special_tokens=False).input_ids
        resp_ids = tokenizer(resp, add_special_tokens=False).input_ids
        ids      = ctx_ids + resp_ids
        labels   = [-100]*len(ctx_ids) + resp_ids
        # truncate
        if len(ids) > MAX_LEN:
            ids    = ids[-MAX_LEN:]
            labels = labels[-MAX_LEN:]
        in_ids.append(ids)
        lbls.append(labels)
    return {"input_ids": in_ids, "labels": lbls}

proc = raw.map(preprocess, batched=False, remove_columns=["messages"])
all_inputs = sum(proc["input_ids"], [])
all_labels = sum(proc["labels"], [])
train_ds   = Dataset.from_dict({"input_ids": all_inputs, "labels": all_labels})

# 9. collate_fn
def collate_fn(batch):
    input_ids = [ex["input_ids"] for ex in batch]
    labels    = [ex["labels"]    for ex in batch]
    enc = tokenizer.pad(
        {"input_ids": input_ids, "labels": labels},
        padding=True,
        return_tensors="pt"
    )
    return enc

# 10. TrainingArguments
training_args = TrainingArguments(
    output_dir="./solar_peft_finetuned",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    learning_rate=3e-4,
    bf16=True,
    logging_steps=50,
    save_steps=500,
    save_total_limit=3,
    report_to="none"
)

# 11. Trainer & 학습
torch.cuda.empty_cache()
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    data_collator=collate_fn
)
trainer.train()

# 12. 저장
model.save_pretrained("./solar_peft_finetuned")
tokenizer.save_pretrained("./solar_peft_finetuned")
print("✅ PEFT 양자화 파인튜닝 완료!")


In [ ]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)
from peft import PeftModel

# 1) Quantization 설정 (4bit)
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# 2) 토크나이저 로드 (파인튜닝 후 저장된 것)
tokenizer = AutoTokenizer.from_pretrained("./solar_peft_finetuned")
tokenizer.pad_token = tokenizer.eos_token

# 3) 베이스 모델 로드 (양자화)
base_model = AutoModelForCausalLM.from_pretrained(
    "upstage/SOLAR-10.7B-v1.0",
    quantization_config=quant_config,
    device_map="auto"
)

# 4) 임베딩 크기 재설정
#    토크나이저의 새 vocab size (32002)에 맞춰 모델 임베딩 확장
base_model.resize_token_embeddings(len(tokenizer))

# 5) PEFT 어댑터 로드
model = PeftModel.from_pretrained(
    base_model,
    "./solar_peft_finetuned",
    device_map="auto"
)
model.eval()

# 6) 이제 모델과 토크나이저를 써서 멀티턴 챗을 돌려 보시면 됩니다.
history = [
    {"role": "system", "content": "당신은 친절하고 정확한 고객상담 챗봇입니다."}
]

while True:
    user_input = input("User: ")
    if user_input.lower() in ("exit","quit"):
        break
    history.append({"role":"user","content":user_input})

    # <|user|> / <|assistant|> 토큰 기반 프롬프트 생성
    prompt = ""
    for turn in history:
        tag = "<|user|>" if turn["role"]=="user" else "<|assistant|>"
        prompt += tag + turn["content"] + tokenizer.eos_token
    prompt += "<|assistant|>"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
        )

    # 새로 생성된 부분만 디코딩
    reply = tokenizer.decode(
        outputs[0][ inputs["input_ids"].shape[-1] : ],
        skip_special_tokens=True
    ).strip()
    print("Assistant:", reply, "\n")
    history.append({"role":"assistant","content":reply})


In [ ]:
# -*- coding: utf-8 -*-
import os
import torch
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, TaskType
import wandb

# — WANDB 설정 (한 번만 로그인)
wandb.login()  

# — 메모리 단편화 완화
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

# — 4bit 양자화 설정
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# — 기저 모델 로드 & LoRA 어댑터 장착
model_name = "naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-1.5B"
base = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto"
)
lora_cfg = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=16,
    lora_alpha=8,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"]
)
model = get_peft_model(base, lora_cfg)
model.config.use_cache = False
model.train()

# — 토크나이저 세팅
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({"additional_special_tokens": ["<|user|>", "<|assistant|>"]})
model.resize_token_embeddings(len(tokenizer), mean_resizing=False)

# — 데이터 로드 & 전처리
raw = load_dataset(
    "json",
    data_files={"train": "asia_mulit.jsonl"},
    split="train"
)

def preprocess(example):
    msgs = [{"role":"system","content":"당신은 친절하고 정확한 고객상담 챗봇입니다."}] + example["messages"]
    input_ids, labels = [], []
    for i, msg in enumerate(msgs):
        if msg["role"] != "assistant":
            continue
        # 컨텍스트
        ctx = ""
        for prev in msgs[:i]:
            tag = "<|user|>" if prev["role"]=="user" else "<|assistant|>"
            ctx += tag + prev["content"] + tokenizer.eos_token
        ctx += "<|assistant|>"
        resp = msg["content"] + tokenizer.eos_token

        ctx_ids  = tokenizer(ctx,  add_special_tokens=False).input_ids
        resp_ids = tokenizer(resp, add_special_tokens=False).input_ids

        ids  = ctx_ids + resp_ids
        lbls = [-100]*len(ctx_ids) + resp_ids

        input_ids.append(ids)
        labels.append(lbls)

    return {"input_ids": input_ids, "labels": labels}

proc = raw.map(preprocess, batched=False, remove_columns=["messages"])
all_in  = sum(proc["input_ids"], [])
all_lbl = sum(proc["labels"], [])
train_ds = Dataset.from_dict({"input_ids": all_in, "labels": all_lbl})

# — collate_fn
def collate_fn(batch):
    enc = tokenizer.pad(
        {"input_ids":[b["input_ids"] for b in batch],
         "labels":[b["labels"]     for b in batch]},
        padding=True,
        return_tensors="pt"
    )
    return enc

# — TrainingArguments (W&B 로깅 포함)
training_args = TrainingArguments(
    output_dir="./hyperclovax1_5B",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=3e-4,
    bf16=True,
    logging_steps=50,
    save_steps=500,
    save_total_limit=3,
    report_to="wandb",
    run_name="hyperclovax",
    project="hyperclovax_project",
    logging_dir="./logs"
)

# — Trainer 실행
torch.cuda.empty_cache()
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    data_collator=collate_fn
)
trainer.train()

# — 저장
model.save_pretrained("./hyperclovax1_5B")
tokenizer.save_pretrained("./hyperclovax1_5B")

print("✅ PEFT 양자화 파인튜닝 완료!")


# solar wandb

In [ ]:
# -*- coding: utf-8 -*-
import os
import torch
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, TaskType
import wandb

# — WANDB 설정 (한 번만 로그인)
wandb.login()  
wandb.init(
    project="solar_project",   # ← 여기에 프로젝트 이름
    name="solar10_7B"               # ← 여기에 런 이름
)

# — 메모리 단편화 완화
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

# — 4bit 양자화 설정
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# — 기저 모델 로드 & LoRA 어댑터 장착
model_name = "upstage/SOLAR-10.7B-Instruct-v1.0" 

base = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto"
)
lora_cfg = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=16,
    lora_alpha=8,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"]
)
model = get_peft_model(base, lora_cfg)
model.config.use_cache = False
model.train()

# — 토크나이저 세팅
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({"additional_special_tokens": ["<|user|>", "<|assistant|>"]})
model.resize_token_embeddings(len(tokenizer), mean_resizing=False)

# — 데이터 로드 & 전처리
raw = load_dataset(
    "json",
    data_files={"train": "asia_mulit.json"},
    split="train"
)

def preprocess(example):
    msgs = [{"role":"system","content":"당신은 친절하고 정확한 고객상담 챗봇입니다."}] + example["messages"]
    input_ids, labels = [], []
    for i, msg in enumerate(msgs):
        if msg["role"] != "assistant":
            continue
        # 컨텍스트
        ctx = ""
        for prev in msgs[:i]:
            tag = "<|user|>" if prev["role"]=="user" else "<|assistant|>"
            ctx += tag + prev["content"] + tokenizer.eos_token
        ctx += "<|assistant|>"
        resp = msg["content"] + tokenizer.eos_token

        ctx_ids  = tokenizer(ctx,  add_special_tokens=False).input_ids
        resp_ids = tokenizer(resp, add_special_tokens=False).input_ids

        ids  = ctx_ids + resp_ids
        lbls = [-100]*len(ctx_ids) + resp_ids

        input_ids.append(ids)
        labels.append(lbls)

    return {"input_ids": input_ids, "labels": labels}

proc = raw.map(preprocess, batched=False, remove_columns=["messages"])
all_in  = sum(proc["input_ids"], [])
all_lbl = sum(proc["labels"], [])
train_ds = Dataset.from_dict({"input_ids": all_in, "labels": all_lbl})

# — collate_fn
def collate_fn(batch):
    enc = tokenizer.pad(
        {"input_ids":[b["input_ids"] for b in batch],
         "labels":[b["labels"]     for b in batch]},
        padding=True,
        return_tensors="pt"
    )
    return enc

# — TrainingArguments (W&B 로깅 포함)
training_args = TrainingArguments(
    output_dir="./solar10_7B",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    num_train_epochs=2,
    learning_rate=3e-4,
    bf16=True,
    logging_steps=50,
    save_steps=500,
    save_total_limit=3,
    report_to="wandb",
    run_name="solar10_7B",
    logging_dir="./logs"
)

# — Trainer 실행
torch.cuda.empty_cache()
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    data_collator=collate_fn
)
trainer.train()

# — 저장
model.save_pretrained("./solar10_7B")
tokenizer.save_pretrained("./solar10_7B")

print("✅ PEFT 양자화 파인튜닝 완료!")


# wandb hugging face

In [ ]:
# finetune_and_push.py

import os
import torch
import wandb
import evaluate
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig,
)
from peft import LoraConfig, get_peft_model, TaskType

# ─── 0. (선택) Hugging Face 인증 ───────────────────────────────────────────────
#  - CLI에서 한 번만: `huggingface-cli login`
#  또는 코드로 직접 토큰 저장:
# from huggingface_hub import HfFolder
# HfFolder.save_token("hf_XXXXXXXXXXXX")  # ← 여기에 본인의 HF token 넣기

# ─── 0.1. Weights & Biases 로그인 & 초기화 ────────────
wandb.login()  
wandb.init(
    project="solar_peft_project_asis",      # ← 원하는 W&B 프로젝트명
    name="solar_10.asia"    # ← 이 실험의 런 이름
)

# ─── 1. 메모리 단편화 완화 ────────────────────────────────────────────────────
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

# ─── 2. 4bit 양자화 설정 ────────────────────────────────────────────────────
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# ─── 3. 기본 모델 로드 및 LoRA 어댑터 부착 ────────────────────────────────────
model_name = "upstage/SOLAR-10.7B-v1.0"
base = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto",
)

lora_cfg = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=16,
    lora_alpha=8,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],
)
model = get_peft_model(base, lora_cfg)

# 캐시 끄기 + gradient checkpointing (메모리 여유시 켜기)
model.config.use_cache = False
# model.gradient_checkpointing_enable()

model.train()

# 멀티 GPU 설정
#if torch.cuda.device_count() > 1:
#   model = torch.nn.DataParallel(model)
    
# ─── 4. 학습 가능 파라미터 확인 (디버그용) ───────────────────────────────────
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"▶︎ Trainable params: {trainable_params:,} / {total_params:,}")

# ─── 5. 토크나이저 세팅 ───────────────────────────────────────────────────────
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({"additional_special_tokens": ["<|user|>", "<|assistant|>"]})
model.resize_token_embeddings(len(tokenizer), mean_resizing=False)

# ─── 6. 데이터 로드 & 전처리 ─────────────────────────────────────────────────
raw = load_dataset(
    "json",
    data_files={"train": "asia_mulit.json"},
    split="train"
)

#MAX_LEN = 256

def preprocess(example):
    # system 프롬프트 + 연속 같은 role 메시지 병합
    msgs = [{"role":"system","content":"""
    당신은 “국립아시아문화전당 공식 고객상담 챗봇”입니다.
아래의 지침을 따라, 멀티턴 대화에서 사용자의 문의에 친절하고 정확하게 답변하세요.

1. **페르소나 & 톤**
   - 정중하면서도 따뜻한 말투를 사용합니다.
   - 고객의 상황에 공감하고, 안심시킬 수 있는 표현을 우선시합니다.
   - 절대로 기계적·딱딱한 어투를 쓰지 않습니다.

2. **도메인 지식 & 제약조건**
   - 회원/비회원 예매 조회, 단체 예약, 취소·환불 규정, 수수료 정책 등
   - 예매 방법(홈페이지·콜센터·현장 매표소)별 차이와 제한 사항
   - 공연명, 공연 일시, 인원, 연락처 등 고객이 제공한 정보를 반드시 반영
   - 최신 운영 시간·정책(예: 티켓 오픈 시간, 취소 수수료 발생 시점)을 정확히 안내

3. **응답 구조 (멀티턴 Q&A)**
   매 턴마다 아래 순서대로 답변을 구성합니다.
   1) **핵심 정보**: 사용자의 직접 질문에 대한 간결한 답변  
   2) **추가 안내**: 필요한 세부 설명이나 주의사항  
   3) **추천 행동**: 다음에 고객이 취해야 할 구체적인 단계를 번호 매겨 제시  

4. **멀티턴 흐름 유지**
   - 고객의 이전 질문을 항상 기억하고 문맥을 반영하세요.
   - 사용자가 추가 정보를 제공하면, 반드시 그 정보(예: “7월 3일 오셀로”)를 답변에 포함합니다.
   - 대화가 길어져도 “핵심→추가 안내→추천 행동” 구조를 매번 유지합니다.
    """}]
    prev_role, buf = None, ""
    for m in example["messages"]:
        role, text = m["role"], m["content"].strip()
        if role == prev_role:
            buf += " " + text
        else:
            if prev_role:
                msgs.append({"role":prev_role, "content":buf})
            buf, prev_role = text, role
    if buf:
        msgs.append({"role":prev_role, "content":buf})
    # assistant 응답마다 context+label 생성
    input_ids, labels = [], []
    for i, m in enumerate(msgs):
        if m["role"] != "assistant": continue
        # context
        ctx = ""
        for prev in msgs[:i]:
            tag = "<|user|>" if prev["role"]=="user" else "<|assistant|>"
            ctx += tag + prev["content"] + tokenizer.eos_token
        ctx += "<|assistant|>"
        # resp
        resp = msgs[i]["content"] + tokenizer.eos_token
        ctx_ids = tokenizer(ctx, add_special_tokens=False).input_ids
        rsp_ids = tokenizer(resp, add_special_tokens=False).input_ids
        ids = ctx_ids + rsp_ids
        lbl = [-100]*len(ctx_ids) + rsp_ids
        # truncate
        # if len(ids) > MAX_LEN:
        #     ids = ids[-MAX_LEN:]
        #     lbl = lbl[-MAX_LEN:]
        input_ids.append(ids)
        labels.append(lbl)
    return {"input_ids": input_ids, "labels": labels}

proc = raw.map(preprocess, batched=False, remove_columns=["messages"])
all_in = sum(proc["input_ids"], [])
all_lb = sum(proc["labels"], [])
train_ds = Dataset.from_dict({"input_ids": all_in, "labels": all_lb})

accuracy_metric  = evaluate.load("accuracy")
bertscore_metric = evaluate.load("bertscore")

def compute_metrics(eval_pred):
    # eval_pred: (logits, labels)
    logits, labels = eval_pred
    # ——— 토큰 단위 Accuracy ———
    preds = logits.argmax(axis=-1)
    mask  = labels != -100
    acc = accuracy_metric.compute(
        predictions=preds[mask].flatten().tolist(),
        references=labels[mask].flatten().tolist()
    )["accuracy"]

    # ——— 시퀀스 디코딩 후 BERTScore(유사도) ———
    # (skip_special_tokens=True 로 본문만)
    pred_strs  = tokenizer.batch_decode(preds,  skip_special_tokens=True)
    label_strs = tokenizer.batch_decode(labels, skip_special_tokens=True)
    bert = bertscore_metric.compute(
        predictions=pred_strs,
        references=label_strs,
        lang="ko"
    )
    # f1 점수 평균
    bert_f1 = sum(bert["f1"]) / len(bert["f1"])

    return {
        "accuracy": acc,
        "bertscore_f1": bert_f1
    }


# ─── 7. collate_fn 정의 ─────────────────────────────────────────────────────
def collate_fn(batch):
    in_ids = [ex["input_ids"] for ex in batch]
    lbs   = [ex["labels"]    for ex in batch]
    return tokenizer.pad(
        {"input_ids": in_ids, "labels": lbs},
        padding=True,
        return_tensors="pt"
    )

# ─── 8. TrainingArguments & Trainer 초기화 ──────────────────────────────────
training_args = TrainingArguments(
    output_dir="./solar_peft_finetuned",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=2,
    learning_rate=3e-4,
    bf16=True,
    logging_steps=50,
    save_steps=500,
    save_total_limit=3,
    report_to = "wandb"
)

torch.cuda.empty_cache()
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    data_collator=collate_fn,
    compute_metrics=compute_metrics
)

# ─── 9. 학습 시작 ────────────────────────────────────────────────────────────
trainer.train()

# ─── 10. 로컬 저장 ──────────────────────────────────────────────────────────
model.save_pretrained("./solar_peft_finetuned")
tokenizer.save_pretrained("./solar_peft_finetuned")
print("✅ 로컬에 모델·토크나이저 저장 완료")

# ─── 11. Hugging Face Hub에 업로드 ───────────────────────────────────────────
from huggingface_hub import HfApi, Repository, HfFolder

# (이미 로그인했다면 아래 줄은 생략 가능)
# HfFolder.save_token("hf_XXXXXXXXXXXX")  # ← HF 토큰을 직접 삽입해도 됩니다

api = HfApi()
repo_id = "seoungji/solar-peft-finetuned"  # ← "hf_계정명/리포명" 으로 수정
# 한 번만 리포지터리 생성 (private=True로 설정 시 비공개)
api.create_repo(name=repo_id.split("/")[-1],
                token=HfFolder.get_token(),
                private=False)

# 로컬 폴더와 원격 리포 연결
repo = Repository(
    local_dir="./solar_peft_finetuned",
    clone_from=repo_id,
    use_auth_token=True
)

# 커밋 & 푸시
repo.push_to_hub(commit_message="Add fine-tuned 4bit LoRA model")
print(f"✅ 모델 업로드 완료: https://huggingface.co/{repo_id}")
